# ADR Classification

In [1]:
import numpy as np
import tensorflow as tf

Set the seed

In [2]:
np.random.seed(42)

In [3]:
import pandas as pd

Import the dataset

In [4]:
df = pd.read_csv('C:/Users/Jagadeesh/Desktop/DS assignment 2 - ADR Clasification/data.csv')

shape of the data frame

In [5]:
df.shape

(23516, 4)

view the first 20 rows

In [6]:
df.head(20)

,UserId,TweetId,Tweet,ADR_label
0,10030778,413205,Intravenous azithromycin-induced ototoxicity.,1
1,10048291,528244,"Immobilization, while Paget's bone disease was...",1
2,10048291,361834,Unaccountable severe hypercalcemia in a patien...,1
3,10082597,292240,METHODS: We report two cases of pseudoporphyri...,1
4,10082597,467101,METHODS: We report two cases of pseudoporphyri...,1
5,10082597,599105,"Naproxen, the most common offender, has been a...",1
6,10082597,259287,RESULTS: A 44-year-old man taking naproxen for...,1
7,10082597,479710,RESULTS: A 44-year-old man taking naproxen for...,1
8,10082597,141130,RESULTS: A 44-year-old man taking naproxen for...,1
9,10082597,787933,RESULTS: A 44-year-old man taking naproxen for...,1


Removing the duplicate tweets

In [7]:
df = df.drop_duplicates(['Tweet'])

In [8]:
df.shape

(20896, 4)

Split data into Training and Testing data

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    df['Tweet'],
    df['ADR_label'],
    test_size = 0.2,
    random_state = 42
    
)

In [11]:
X_train.shape

(16716,)

# Build the Tokenizer

In [12]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

Most frequent number of words

In [13]:
top_words = 25000

In [14]:
t = Tokenizer(num_words = top_words)

In [15]:
t.fit_on_texts(X_train.tolist())

# Preparing Training and Testing Data

Get the word index for each of the word in the review

In [16]:
X_train = t.texts_to_sequences(X_train.tolist())

In [17]:
X_test = t.texts_to_sequences(X_test.tolist())

# Pad Sequences

In [18]:
from tensorflow.python.keras.preprocessing import sequence

In [19]:
max_input_length = 150

In [20]:
X_train = sequence.pad_sequences(X_train, maxlen = max_input_length, padding = 'post')

In [21]:
X_test = sequence.pad_sequences(X_test, maxlen = max_input_length, padding = 'post')

# Defining embedding layer configuration

In [22]:
embedding_vector_length = 75

# Build the Graph

In [23]:
from tensorflow.python.keras.models import Sequential

In [24]:
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten

In [25]:
model = Sequential()

Add embedded layer

In [26]:
model.add(
        Embedding (top_words+1,
                  embedding_vector_length,
                  input_length= max_input_length)
            )

In [27]:
model.add(Flatten())

In [28]:
model.add(Dense(200,activation = 'relu'))

In [29]:
model.add(Dense(100,activation = 'relu'))

In [30]:
model.add(Dense(60,activation = 'relu'))

In [31]:
model.add(Dense(30,activation = 'relu'))

In [32]:
model.add(Dense(1,activation = 'sigmoid'))

In [33]:
model.compile(optimizer = 'adam', loss ='binary_crossentropy',metrics = ['accuracy'])

# Execute the Graph

In [ ]:
model.fit(X_train, y_train,
         epochs = 12,
         batch_size = 128,
         validation_data = (X_test, y_test)
         )

C:\Users\Jagadeesh\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 16716 samples, validate on 4180 samples
Epoch 1/12
16716/16716 [==============================] - 28s 2ms/step - loss: 0.5725 - acc: 0.7881 - val_loss: 0.5054 - val_acc: 0.7983

In [56]:
predictions = model.predict(X_train)
 
score = model.evaluate(X_train,y_train,verbose=0)
print('Train loss:', score[0])
print('Train accuracy:', score[1])



Train loss: 0.0052088276643798975
Train accuracy: 0.9994615936826992


In [57]:
predictions = model.predict(X_test)
 
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.5923214689182322
Test accuracy: 0.8751196173389563
